# CSPB 3202 Final Project #

Tyler Kinkade, Tyler.Kinkade@colorado.edu    

_All rights reserved_

GitHub: [https://github.com/jaki9292/rl-project](https://github.com/jaki9292/rl-project)

## Overview ##

This write-up reports on a small project to compare the effectiveness of artificial agent algorithms ranging in sophistication from naive to deep reinforcement learning (Russell & Norvig, 2022; Sutton & Barto, 2018) to successfully land a virtual "lunar lander" in a Gymnasium (2022) model environment.

The report is divided into three main sections: approach, results, and discussion.

## Approach ##

My approach to the problem is described in the following environment, models, and methods subsections.

### Environment ###

The lunar lander environment (pictured below) simulates rocket trajectory physics with the primary aim of landing a lunar lander on a central landing pad (marked by two flags) by means of turning its three rockets thrusters (left, right, and main) on or off. The agent operating the lander is rewarded for landing slowly, in an upright position, on both legs, on or near the landing pad and penalized otherwise. Fuel is unlimited, but a penalty is given for each time an engine fires. Episodes are truncated after 1000 time-steps. Scores of 200 points or more are considered a solution. 

The state space is described by an 8-vector comprised of the lander coordinates $(x,y)$, its linear velocities $(v_x,v_y)$ in the $x$ and $y$ directions, its angle $(\theta)$, its angular velocity $(\omega)$, and two Boolean variables $(l,r)$ that encode whether the left and right legs are in contact with the ground. The environment has both discrete and continuous versions, but only the discrete version is used here, in order to include simple models in the comparison. The environment gravity, wind power, and turbulence can also be specified. The starting state is the top center of the space with a random force applied to the lander. The termination state occurs when the lander stops moving or moves outside the frame. The action space is comprised of 4 discrete actions: do nothing, fire left thruster, fire main down thruster, or fire right thruster.

In [ ]:
# Display Lunar Lander GIF
from IPython.display import Image
Image(url= "https://gymnasium.farama.org/_images/lunar_lander.gif")

In [ ]:
# Install dependencies
# Packages preinstalled on Google Colab:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install torch

# Packages not pre-installed on Google Colab:
# !pip install gymnasium
# !pip install gymnasium[box2d]
# !pip install moviepy
# !pip install torchinfo

In [ ]:
# Display environment parameters
# References: 
# https://gymnasium.farama.org/api/env/
# https://gymnasium.farama.org/environments/box2d/lunar_lander/

import gymnasium as gym

# Initialize environment
env = gym.make("LunarLander-v2", 
               continuous = False,      # Discrete version
               gravity = -10.0, 
               enable_wind = False, 
               wind_power = 0.0, 
               turbulence_power = 0.0, 
               render_mode="rgb_array") # Render for machine

# Reset environment with random number generator seed for reproducibility
state, info = env.reset(seed = 21)

print("Environment metadata:\n", env.metadata)
print("\nState space:\n", env.observation_space)
print("\nAction space:\n", env.action_space)
print("\nStarting state:\n", state)

The table below summarizes the descriptive statistics for 100 randomly sampled initial states for this environment. We can see that the initial linear and angular velocities vary more than the position and angular orientation.

In [ ]:
import pandas as pd

# Determine average starting state from 100 random samples
starting_states = []
for _ in range(100):
    starting_state, info = env.reset()
    starting_states.append(starting_state)

features = ['x','y','vx','vy','angle','ang_vel','left_leg','right_leg']
starting_states_df = pd.DataFrame(starting_states, columns = features)
starting_states_df.describe()

### Models ###
Four models are compared in this project: an agentless lunar lander, a random agent, a simple reflex agent, and a deep Q-network agent. 

#### Agentless Lunar Lander ####
An agentless lunar lander takes no action and is at the complete mercy of its environment. Because the starting state is directly above the landing pad, this might not be as terrible a strategy as it initially seems. The lander will simply fall to the ground in a trajectory determined by the physics of the environmental forces and the initial force placed upon it.

#### Random Agent ####

A random agent selects any of the possible actions uniformly randomly at each step without regard to the environment. The animation above is an example of such an agent and appears to be a poor strategy. Is this a worse strategy than an agentless lander though?

#### Simple Reflex Agent ####

A simple reflex agent can only react in a fixed way to sensory input but does not retain any memory of the past and, as a result, cannot learn (Russell & Norvig, 2022). As explained in the environment section, the initial lunar lander position is generally centered above the landing pad, but there is wide variation in initial linear and angular velocity. Thus, the primary task is counteracting any horizontal velocity while at the same time, avoiding turning the ship on its side. Therefore, there is a balance to be achieved, and the programmer must select appropriate threshold constants for the algorithm. Depending on the algorithm and constants, this could potentially be a better strategy than doing nothing or acting randomly.

#### Deep Q-Network Agent ####

A deep Q-network (DQN; Mnih et al., 2015, 2016) agent is a deep-learning neural network approximation of a Q-learning agent (Russell & Norvig, 2022; Sutton & Barto, 2018). A Q-learning agent alternates between learning through exploration and exploiting what it has learned by choosing the action with maximal value for each state. Although this is obviously better than the relatively ignorant reflex agent, Q-learning agents can quickly become computationally expensive and slow to learn in large and complex environments (Russell & Norvig, 2022; Sutton & Barto, 2018). 

Approximate Q-learning agents address this problem by approximating the Q-value of actions in particular states with a weighted feature function based on states and actions (Russell & Norvig, 2022; Sutton & Barto, 2018). However, in the environment in this project, we only have access to the current features and do not know how any particular action will influence those features. 

Fortunately, the universal approximation theorem tells us that we can approximate any function with a neural network of at least two layers  (Russell & Norvig, 2022). Thus, we can approximate a Q-value function with a DQN by recording a log of previous attempts (i.e., replay buffer) to train the model (Paszke & Towers, 2017). In other words, the agent remembers and learns from its experiences. In addition, this allows us to sample randomly from the buffer to reduce variance (Morales, 2020). Value gradient clipping will be used to 

For this project, the DQN consists of three fully-connected neural network layers. A non-linear activation function is placed between the layers to prevent the model from collapsing into a one-layer linear model. Here, a rectified linear unit (ReLU) function is selected for this because of its computational simplicity and its consistent, demonstrated effectiveness in previous studies (Goodfellow et al., 2016; Karpathy, 2016). The input to the network are the eight state features and the output is a vector of approximate Q-values for each of the four possible actions given the input state. 

It is worth noting that image-based convolutional neural networks are frequently used for DQNs (e.g., Mnih et al., 2015). However, I am interested here in solving the lunar landing problem from the perspective of the sensory information available to an agent within the lunar lander, rather than an outside observer.

Another design choice is how often the agent should explore versus how often it should exploit what it has learned thus far. For this project, I will use an epsilon greedy policy which prioritizes learning at the beginning but gradually shifts to prioritizing achieving the highest rewards by selecting the policy it has learned thus far.

To train the model, I will use the Huber loss function (Huber, 1964), which is more robust to noisy data than mean-squared error for training the model (Paszke & Towers, 2017).For optimization, I will use the AdamW optimization algorithm (Loshchilov & Hutter, 2019). Unlike the stochastic gradient descent, Adam optimization maintains and adapts separate learning rates for each parameter (Brownlee, 2017). AdamW improves upon this by decoupling weight decay from the gradient update (Loshchilov & Hutter, 2019). Gradient clipping will be used to avoid exploding gradients (Goodfellow et al., 2016).

A potential problem with training DQNs is that parameter updates cause the targets to shift leading to divergence away from an optimal policy (Morales, 2020). To counteract this two sets of parameters are used: an online set which is updated every iteration and a target set which is updated less frequently, or at a specified rate $\tau$, and whose Q-values are used to calculate the loss and train the online set. This allows the parameters to stabilize.

It is expected that the DQN model will perform better than the first three; however, it is still possible that some difficulties might arise in creating a neural network which converges to an optimal policy.

### Methods ###

The first three models do not require training, so they will be run for 100 episodes because their average score will not differ significantly after even more episodes. The DQN model will likely require training for at least 500 episodes as well as hyperparameter tuning. For all models, the single-episode truncation will be reduced to 600 time-steps in order to decrease the training time.

The scores over each of the episodes will be plotted and compared. Visualizing each model's performance over successive episodes permits comparison of how each agent progressed in learning how to solve the model. As Machado and his colleagues (2018) point out, this is preferable to single metrics, such as maximum or average scores which can conceal important distinctions. For example, we can discover whether an agent achieved a maximum only once, but performed poorly most of the time. Similarly, an average score can conceal unstable, fluctuating performance. Ideally, we would like to see an agent learning quickly and then stably maintaining high scores above the solution threshold of 200 points.

Applying this methodology, we will be able to explore the advantages and disadvantages of the various models and also demonstrate how a solution to the problem can be explored through the modfication of features and comparison of models.

In the following code blocks, I construct the Python classes required to implement each of the agents.

#### Agentless Lunar Lander ####
An agentless lunar lander which never fires any of its thrusters:

In [ ]:
# Define general lunar lander class
#
# Adapted from: 
# CSPB 3202 Gym tutorial (https://applied.cs.colorado.edu/mod/folder/view.php?id=43136)
# https://gymnasium.farama.org/
# https://gymnasium.farama.org/api/utils/#save-rendering-videos
# https://gymnasium.farama.org/environments/box2d/lunar_lander/
# https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py
# https://gymnasium.farama.org/tutorials/training_agents/FrozenLake_tuto/
# https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
#
# Additional references:
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.isinteractive.html
# https://www.statology.org/matplotlib-inline/
# https://www.tutorialspoint.com/how-to-check-that-pylab-backend-of-matplotlib-runs-inline
# https://learnpython.com/blog/average-in-matplotlib/

from gymnasium.utils.save_video import save_video
import numpy as np
from IPython.utils.capture import capture_output
import matplotlib
import matplotlib.pyplot as plt

# Display graphs inline
%matplotlib inline

# Check if inline display is possible
is_ipython = 'inline' in matplotlib.get_backend()

# If Jupyter notebook
if is_ipython:
    from IPython import display

class LunarLander:
    '''
    General lunar lander class 
    subject entirely to its environment
    '''   
    def __init__(self, name = "Agentless Lander", step_limit = 600):
        '''
        Initialize agent
        '''
        self.name = name
        self.prefix = "-".join(name.split()).lower() # filename prefix
        self.step_limit = step_limit  # max. allowed steps/episode

        # Initialize environment
        self.env = gym.make("LunarLander-v2", 
               continuous = False,           # Discrete version
               gravity = -10.0, 
               enable_wind = False, 
               wind_power = 0.0, 
               turbulence_power = 0.0, 
               render_mode="rgb_array_list") # Render for machine

        # Define actions
        self.NOOP = 0   # No operation (i.e., do nothing)
        self.LEFT = 1   # Fire left orientation engine
        self.MAIN = 2   # Fire main downward engine
        self.RIGHT = 3  # Fire right orientation engine

        # Reset environment with random number generator seed for reproducibility
        self.state, self.info = self.env.reset(seed = 21)
        
        # Set environment dimensions
        self.features_size = len(self.state)
        self.actions_size = env.action_space.n
  
        # Initialize data logs
        self.scores = []
        self.steps = []
        self.policy = []

        # Initialize total iteration step count
        self.total_steps = 0

        # Set learning rate scheduler to null
        self.scheduler = None

    def select_action(self, state):
        '''
        Select action given state
        '''
        # Do nothing
        return self.NOOP

    def episode(self):
        '''
        Complete episode in given environment
        Return score, steps count, and actions log
        '''
        # Initialize rewards total
        score = 0.0

        # Initialize steps count
        episode_steps = 0

        # Initialize actions list
        actions = []

        # Start new episode  
        state, info = self.env.reset()

        # Until end of episode
        while True:
            # Get action based on current state
            action = self.select_action(state)

            # Obtain next state, reward, terminated status, truncated status, 
            # and environment info for given action
            next_state, reward, terminated, truncated, info = self.env.step(action)
            
            # Accumulate reward total
            score += reward

            # Increment episode steps
            episode_steps += 1

            # Increment overall step count
            self.total_steps += 1

            # Record action
            actions.append(action)

            # If number of steps has been surpassed or end of episode
            if (episode_steps > self.step_limit) or terminated or truncated:
                break  # out of loop
            
            # Advance to next state 
            state = next_state

        return score, episode_steps, actions
    
    def iterate(self, n = 100,      # Number of iterations
                verbose = False,    # Print results for each episode
                video = False,      # Output videos of (k^3)th episodes
                plot = True):       # Display plot updated every episode
        '''
        Attempt task for n episodes
        '''
        # Enable interactive plotting
        plt.ion()

        # Counter for indexing start of videos
        video_step_start = 0

        # Repeat n times
        for episode in range(n):
            # Complete one episode
            score, episode_steps, actions = self.episode()
            
            # Log scores, steps, policy
            self.scores.append(score)
            self.steps.append(episode_steps)
            self.policy.append(actions)

            # If learning rate scheduler is specified
            if self.scheduler:                        
                # Increment learning rate scheduler count
                # Scheduler will perform learning rate decrement
                # after kth episode specified in lr_period parameter 
                self.scheduler.step()
            
            if plot:
                # Update training plot
                self.plot_scores()

            if verbose:
                # Report result
                print(f"Episode {episode} score: {score}")

            if video:
                # Suppress MoviePy stdout
                # https://stackoverflow.com/a/35624406/14371011
                with capture_output() as captured:
                    # Save mp4 of every (k^3)th episode (0, 1, 8, 27, ...)
                    save_video(self.env.render(),
                               video_folder = "videos",
                               name_prefix = self.prefix,
                               fps = env.metadata["render_fps"],
                               step_starting_index = video_step_start,
                               episode_index = episode)

            # Advance starting step index for videos
            video_step_start = episode_steps + 1           
        
        if plot:
            # Plot final result
            self.plot_scores(show_result = True)
            plt.show() # Display plot

        # Disable interactive plotting
        plt.ioff() 
    
    def score_stats(self, verbose = False):
        '''
        Return mean, max, and min of scores if they exist 
        '''
        # If scores exist
        if (len(self.scores) > 0):
            # Calculate average, max, and min score
            average = np.average(self.scores)
            max = np.max(self.scores)
            min = np.min(self.scores)
            
            # Calculate average episode length
            avg_length = np.average(self.steps)

            if verbose:
                # Report average, max, and min
                print(f"\nAverage score: {average}") 
                print(f"Maximum score: {max}")
                print(f"Minimum score: {min}")
                print(f"Average episode length: {avg_length}")

            return average, max, min, avg_length
        else: 
            print("No scores have been calculated")
            return None, None, None
        
    def plot_scores(self, show_result = False):
        '''
        Dynamically plot episode scores
        Plot running average at conclusion 
        '''
        # Initialize plot
        plt.figure(1)

        # If not final plot
        if not show_result:
            # Clear current figure
            plt.clf()

        # Plot solution threshold
        plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  

        # Plot episode-by-episode scores
        plt.plot(self.scores, label = self.name)
        plt.title(self.name)
        plt.xlabel("Episode")
        plt.ylabel("Score")

        # If final plot
        if show_result:
            # Plot running average

            # Span for running average
            window = 25
            
            # Initialize list with one fewer NaNs than window
            averages = [np.nan] * (window - 1)

            # Calculate running average
            for idx in range(len(self.scores) - window + 1):
                averages.append(np.mean(self.scores[idx : idx + window]))

            # Plot running average
            plt.plot(averages, color = "k", label = "Running Average")

        plt.legend()

        # Pause to allow plot to update
        plt.pause(0.001)

        # If Jupyter notebook
        if is_ipython:
            if not show_result:
                # Display plot
                display.display(plt.gcf())
                # Clear plot when done
                display.clear_output(wait = True)
            else:
                # Display plot
                display.display(plt.gcf())

In [ ]:
# Video clips are saved in cubic intervals of episodes, so for reference, the first 41 cubes (from https://oeis.org/A000578) are:
cubes = [0, 1, 8, 27, 64, 125, 216, 343, 512, 729, 1000, 1331, 1728, 2197, 2744, 3375, 4096, 4913, 5832, 6859, 8000, 9261, 10648, 12167, 13824, 15625, 17576, 19683, 21952, 24389, 27000, 29791, 32768, 35937, 39304, 42875, 46656, 50653, 54872, 59319, 64000]
len(cubes)

#### Random Agent ####
An agent which selects any of the four possible actions randomly at each time step:

In [ ]:
# Define random agent class

class RandomAgent(LunarLander):
    '''
    Inherits from lunar lander class
    and acts randomly
    '''
    def __init__(self, name = "Random Agent"):
        LunarLander.__init__(self, name)

    def select_action(self, state):
        '''
        Randomly select action
        '''
        # Randomly select action from action space
        action = self.env.action_space.sample()

        return action

#### Simple Reflex Agent ####

This is an agent which can fire its side thrusters in reaction to its current horizontal and angular velocity in relation to threshold constants chosen by the programmer.

In [ ]:
class ReflexAgent(LunarLander):
    '''
    Inherits from lunar lander class, but 
    selects actions according to horizontal 
    and angular position and angular velocity
    '''
    def __init__(self, name = "Reflex Agent"):
        LunarLander.__init__(self, name)

    def select_action(self, state):
        '''
        Select action according to horizontal and
        angular position and angular velocity
        '''
        # Set tolerances
        horiz_vel_tolerance = 0.4
        ang_vel_tolerance = 0.001

        # Horizontal velocity
        vx = state[3]

        # Angular velocity
        ang_vel = state[5]

        # If leftward horizontal velocity
        if ((vx < -horiz_vel_tolerance) and
            # and clockwise velocity
            (ang_vel > ang_vel_tolerance)):

            # Then, fire thruster to move right
            action = self.RIGHT
        
        # If rightward horizontal velocity
        elif ((vx > horiz_vel_tolerance) and
              # If counter-clockwise velocity
              (ang_vel < -ang_vel_tolerance)):
            
            # Then, fire thruster to move left
            action = self.LEFT

        else:
            # Do nothing
            action = self.NOOP
        
        return action 

#### Deep Q-Network Agent ####

A deep Q-network agent has the capacity to remember and learn from its previous experiences using a neural network.

##### Replay Memory and Deep Q-Network #####
First, we must define a buffer to remember previous experiences and a neural network for the agent to use for learning.

In [ ]:
# Define Replay Memory Buffer and Deep Q-Network
# Adapted from: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
# Additional references:
# https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch
# https://pytorch.org/docs/stable/notes/randomness.html

import random
from collections import namedtuple, deque

# Named tuple data structure to encode a single state transition
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# Buffer class to hold and randomly sample recent transitions
class ReplayMemory(object):

    def __init__(self, capacity):
        '''
        Initialize buffer with empty double-ended queue
        of size "capacity"
        '''
        self.memory = deque([], maxlen = capacity)

        # Set random seed for reproducibility
        random.seed(21)

    def push(self, *args):
        '''
        Add a transition to buffer
        '''
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        '''
        Randomly sample batch of transitions from buffer
        '''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''
        Returns number of transitions in buffer
        '''
        return len(self.memory)

import torch
import torch.nn as nn
import torch.nn.functional as F

# Deep Q-Network
class DQN(nn.Module):

    def __init__(self, features_size, actions_size):
        '''
        Initialize neural network
        '''
        super(DQN, self).__init__()

        # First fully-connected layer: 
        # Input dimensions: 1 x no. of state feature
        # Output dimensions: 1 x 128
        self.fc1 = nn.Linear(features_size, 128)

        # Second fully-connected layer: 
        # Input: 1 x 128; Output: 1 x 128
        self.fc2 = nn.Linear(128, 128)

        # Final fully-connected layer: 
        # Input: 1 x 128; 
        # Output: 1 x no. of possible actions
        self.fc3 = nn.Linear(128, actions_size)

    def forward(self, state):
        '''
        Forward propagation function
        Input: State features (e.g., position, velocity, ...)
        Output: tensor of action values
        '''
        # First layer followed by ReLU activation function
        z = F.relu(self.fc1(state))
        
        # Second layer followed by ReLU activation function
        z = F.relu(self.fc2(z))
        
        # Final output layer
        return self.fc3(z)

##### DQN Agent ####
Now we can define the agent.

In [ ]:
# Define DQN agent
# References:
# Adapted from: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
# Additional references:
# https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch
# https://pytorch.org/docs/stable/notes/randomness.html
# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
# https://pytorch.org/docs/stable/generated/torch.max.html
# https://stackoverflow.com/questions/19339/transpose-unzip-function-inverse-of-zip/19343
# https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html
# https://machinelearningmastery.com/using-learning-rate-schedule-in-pytorch-training/


import torch.optim as optim
import math

class DQNAgent(LunarLander):
    '''
    Inherits from lunar lander class, but 
    iteratively learns from experience to approximate 
    Q-values from state features with a neural network
    '''
    def __init__(self, 
                 name = "DQN Agent", 
                 alpha = 0.1,          # initial optimizer learning rate
                 epsilon = (1.0,       # epsilon-greedy max
                            0.01,      # epsilon-greedy min
                            0.001),    # exponential decay rate (smaller = slower)
                 gamma = 0.99,         # discount rate
                 tau = 0.005,          # target network update rate
                 lr_period = 100,      # episodes between learning rate decreases
                 lr_decay = 0.1,       # learning rate decay
                 batch_size = 128,     # replay buffer batch size
                 buffer_size = 1000000 # replay buffer size (no. of transitions)
                 ):
        '''
        Initialize agent hyperparameters and data structures
        '''
        LunarLander.__init__(self, name)
        self.alpha = alpha
        self.eps_max, self.eps_min, self.eps_decay = epsilon
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.total_steps = 0

        # Set random seeds for reproducibility
        torch.manual_seed(21)
        random.seed(21)

        # Set device to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:",self.device)

        # Initialize online and target deep Q-networks 
        # and move to selected device
        self.online_net = DQN(self.features_size, self.actions_size).to(self.device)
        self.target_net = DQN(self.features_size, self.actions_size).to(self.device)
        self.target_net.load_state_dict(self.online_net.state_dict())

        # Initialize optimizer
        self.optimizer = optim.AdamW(self.online_net.parameters(), 
                                lr = self.alpha, # learning rate
                                amsgrad = True)  # Reddi et al., 2019 variant
        
        # Initialize learning rate scheduler
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, lr_period, lr_decay)

        # Initialize Huber loss function
        self.criterion = nn.SmoothL1Loss()

        # Initialize replay memory buffer 
        # with maximum capacity of buffer_size transitions
        self.memory = ReplayMemory(buffer_size)

    def select_action(self, state):
        '''
        Epsilon-greedy action selection with online DQN
        '''       
        # Calculate exponentially decaying epsilon-greedy threshold
        # (as number of steps becomes large, threshold becomes minimum)
        eps_threshold = (self.eps_min + (self.eps_max - self.eps_min) 
                            * math.exp(-self.total_steps * self.eps_decay))
        
        # Increment total steps
        self.total_steps += 1

        # Epsilon-greedy algorithm
        # If random number [0,1) is greater than epsilon threshold
        if (random.random() > eps_threshold):
            # Without backward propagation 
            with torch.no_grad():
                # Select action with maximum expected reward 
                # from online network

                # Get index of maximum expected reward
                max_value_index = self.online_net(state).max(1)[1]

                # Get corresponding action
                best_action = max_value_index.view(1,1)

                return best_action
            
        else:
            # Randomly select action from action space
            # and return as tensor
            return torch.tensor([[env.action_space.sample()]], 
                                device = self.device, dtype = torch.long)
    
    def optimize_model(self):
        '''
        Perform single step of optimization on batch of transitions
        '''
        # If fewer transitions in replay buffer than batch size
        if len(self.memory) < self.batch_size:
            # Can't run batch yet
            # so, just keep collecting transitions
            return
        
        else:
            # Randomly select batch of transitions from
            # replay buffer
            transitions = self.memory.sample(self.batch_size)

            # Transpose from batch array of transitions 
            # to transition of batch-arrays
            batch = Transition(*zip(*transitions))

            # Mask to select only non-final states 
            # (i.e., states whose next state is not null)
            non_final_mask = torch.tensor(tuple(map(lambda s: 
                                                    s is not None, 
                                                    batch.next_state)), 
                                                    device = self.device, 
                                                    dtype = torch.bool)

            # Concatenate the non-null next states
            non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
            
            # Concatenate states, actions, and rewards
            state_batch = torch.cat(batch.state)
            action_batch = torch.cat(batch.action)
            reward_batch = torch.cat(batch.reward)

            # Calculate online network Q-values for each state and action, Q(s,a) 
            online_Q_values = self.online_net(state_batch).gather(1, action_batch)

            # Initialize tensor of next state expected rewards, V(s') = 0
            target_next_state_values = torch.zeros(self.batch_size, device = self.device)
                    
            # Without back propagation
            with torch.no_grad():
                # Calculate target network expected reward (i.e., value) 
                # for each next state, V(s') (post-terminal states will have have value 0)
                target_next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1)[0]
                           
            # Calculate target network expected Q values 
            target_expected_Q_values = ((target_next_state_values * self.gamma) 
                                            + reward_batch)

            # Calculate loss by comparing online and target network Q values
            loss = self.criterion(online_Q_values, 
                            target_expected_Q_values.unsqueeze(1))

            # Reset optimizer gradients
            self.optimizer.zero_grad()

            # Back propagation
            loss.backward()

            # Perform gradient clipping by value to prevent exploding gradients
            torch.nn.utils.clip_grad_value_(self.online_net.parameters(), 100)
            
            # Adjust learning weights
            self.optimizer.step()

    def episode(self):
        '''
        Complete episode in given environment
        Call DQN optimizer
        Update online and target networks
        Return score, steps count, and actions log
        '''
        # Initialize rewards total
        score = 0.0

        # Initialize steps count
        episode_steps = 0

        # Initialize actions list
        actions = []

        # Start new episode  
        state, info = self.env.reset()

        #Convert to tensor
        state = torch.tensor(state, 
                            dtype = torch.float32, 
                            device = self.device).unsqueeze(0)

        # Until end of episode
        while True:
            # Get action based on current state
            action = self.select_action(state)

            # Obtain next state, reward, terminated status, truncated status, 
            # and environment info for given action
            next_state, reward, terminated, truncated, info = self.env.step(action.item())

            # Accumulate reward total
            score += reward

            # Convert reward to tensor
            reward = torch.tensor([reward], device = self.device)

            # Increment episode steps
            episode_steps += 1

            # Increment overall step count
            self.total_steps += 1

            # Record action
            actions.append(action)

            # If action taken leads to terminated state
            if terminated or truncated:
                # Next state is null
                next_state = None
            else:
                # Convert next state to tensor
                next_state = torch.tensor(next_state, 
                                        dtype = torch.float32, 
                                        device = self.device).unsqueeze(0)

            # Store transition in replay buffer
            self.memory.push(state, action, next_state, reward)
         
            # Advance to next state 
            state = next_state

            # Perform one step of optimization on online network
            self.optimize_model()            

            # Get target and online network state dicts
            target_net_state_dict = self.target_net.state_dict()
            online_net_state_dict = self.online_net.state_dict()

            # For each parameter in the online network state dict
            for param in online_net_state_dict:
                # Update target network weights at rate tau
                # target parameter <-- tau(online parameter) + (1-tau)(target parameter)
                target_net_state_dict[param] = (online_net_state_dict[param]*self.tau 
                                                + target_net_state_dict[param]*(1-self.tau))
            
            # Load updated target network state dict
            self.target_net.load_state_dict(target_net_state_dict)

            # If max. number of steps has been surpassed or end of episode
            if (episode_steps > self.step_limit) or terminated or truncated:
                break

        return score, episode_steps, actions


## Results ## 

In this section, I compare the five different models: an agentless lunar lander, a random agent, a simple reflex agent, and a DQN agent.

### Agentless Lunar Lander ###
First, we consider the agentless lunar lander which never fires any of its thrusters and is therefore subject entirely to its environment:

In [ ]:
# Instantiate agentless lunar lander
agentless_lander = LunarLander()

# Maintain list of agent objects
agents = [agentless_lander]

episodes = 10

# Iterate over selected number of episodes
agentless_lander.iterate(episodes, verbose = False, video = True, plot = True)

# Print average, max, min
average, max, min, avg_length = agentless_lander.score_stats(verbose = True)

[Link to the video of the agentless lander in the 64<sup>th</sup> episode](videos/agentless-lander-episode-64.mp4)

In the above graph, we can see that an agentless lunar lander is not sufficient to solve this problem. In the video linked above, we can see the lunar lander simply falling in a slightly curved trajectory toward the ground.

### Random Agent ###

Next, we consider the random agent which selects an action randomly.

In [ ]:
# Instantiate lunar lander random agent
rand_agent = RandomAgent()

# Add to list of agents
agents.append(rand_agent)

# Iterate over episodes
rand_agent.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min, avg_length = rand_agent.score_stats(verbose = True)

[Link to the video of the random agent in the 64<sup>th</sup> episode](videos/random-agent-episode-64.mp4)

In [ ]:
def plot_comparison(agents):
    '''
    Plot scores of multiple agent objects
    '''
    # Initialize plot
    plt.figure(1)
    plt.ylim([-600,300])

    # Plot solution threshold
    plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  

    # Initialize list of agent names
    names = []

    # Iterate over agents list
    for agent in agents:
        # Plot agent's scores
        plt.plot(agent.scores, label = agent.name)
        names.append(agent.name)

    plt.title(", ".join(names))
    plt.xlabel("Episode")
    plt.ylabel("Score")
    plt.legend()
    plt.show();

In [ ]:
# Plot agentless lander and random agent scores over all episodes
plot_comparison(agents)

As the above graph shows, doing nothing is better than acting randomly in this environment. The agentless lunar lander is relatively more consistent and successful compared to the random agent and also has a slightly higher average score. Obviously, neither learn anything nor come remotely close to consistently reaching the solution threshold, as expected.

### Simple Reflex Agent ###

Next, we examine the agent which can reflexively fire its side engines in reaction to its current horizontal and angular velocity mediated by pre-selected threshold constants. Choosing ideal threshold constants requires tedious trial and error on the part of the programmer, rather than the agent. Thus, only the programmer is learning in this scenario.

In [ ]:
# Instantiate lunar lander reflex agent
rflx_agent = ReflexAgent()

# Add to list of agents
agents.append(rflx_agent)

# Iterate over episodes
rflx_agent.iterate(episodes, verbose = False, video = True)

# Print average max, min
average, max, min, avg_length = rflx_agent.score_stats(verbose = True)

[Link to the video of the reflex agent in the 64<sup>th</sup> episode](videos/reflex-agent-episode-64.mp4)

In [ ]:
# Plot comparison of agents
plot_comparison(agents)

In the above graph, we can see that the reflex agent is only slightly more consistent and successful than the agentless lunar lander but still not even close to solving the problem. Additionally, it is clearly not learning, as expected. Landing a lunar rover appears to be more difficult than firing the left and right thrusters based on horizontal and angular velocity. It might be possible to achieve a higher score by comparing various algorithms and parameters through trial and error, but this would be a tedious process, so we would like to find a more expedient way to find the best policy to follow.

### Deep Q-Network Agent ###

Next, we examine the performance of a remembering, learning deep Q-network agent. We'll start with the following hyperparameters.

In [ ]:
# Hyperparameters
name = "DQN Agent 1"
alpha = 0.1             # initial optimizer learning rate
epsilon = (0.99,        # epsilon-greedy max (starting value)
        0.01,           # epsilon-greedy min (ending value)
        0.001)          # exponential decay rate (smaller = slower)
gamma = 0.99            # discount rate
tau = 0.005             # target network update rate
lr_period = 100         # episodes between learning rate decreases
lr_decay = 0.1          # learning rate decay
batch_size = 128        # replay buffer batch size
buffer_size = 1000000   # replay buffer size (no. of transitions)

Let's confirm the neural network architecture and estimated computation requirements.

In [ ]:
# Instantiate lunar lander DQN agent
dqn_agent1 = DQNAgent(name, alpha, epsilon, gamma, tau, lr_period, lr_decay, batch_size, buffer_size)

# View model summary
# https://github.com/TylerYep/torchinfo
# https://discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda/180/7

print("Model parameters on CUDA?",next(dqn_agent1.online_net.parameters()).is_cuda)
print(dqn_agent1.online_net)

from torchinfo import summary
import warnings

warnings.filterwarnings("ignore")

summary(dqn_agent1.online_net, 
        input_size = (dqn_agent1.batch_size, dqn_agent1.features_size))

warnings.filterwarnings("default")

In [ ]:
# Train DQN agent

# If GPU device is available
if torch.cuda.is_available():
    episodes = 1000
    print("Device: cuda")
else:
    episodes = 600
    print("Device: cpu")

# Train for specified number of episodes
dqn_agent1.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min, avg_length = dqn_agent1.score_stats(verbose = True)

[Link to the video of the first DQN agent in the 512<sup>th</sup> episode](videos/dqn-agent-1-episode-512.mp4)

This agent seems to have already begun settling on a policy that resulted in scores worse than the reflex agent. If we look at the video linked above, we see that the agent seems to be trying to hover at the top center of the screen to accumulate rewards by being close to center on the horizontal axis (albeit far from the landing pad at the origin and not taking advantage of the 100-point landing reward). It has learned a strategy, but not the strategy we want it to learn.

Let's give the agent some more incentive to stop hovering in the air and get to the business of landing by lowering the discount rate. This will lead to diminishing returns for just hovering.

In [ ]:
# Hyperparameters
name = "DQN Agent 2"
alpha = 0.1             # initial optimizer learning rate
epsilon = (0.99,        # epsilon-greedy max (starting value)
        0.01,           # epsilon-greedy min (ending value)
        0.001)          # exponential decay rate (smaller = slower)
gamma = 0.8             # discount rate
tau = 0.005             # target network update rate
lr_period = 100         # episodes between learning rate decreases
lr_decay = 0.1          # learning rate decay
batch_size = 128        # replay buffer batch size
buffer_size = 1000000   # replay buffer size (no. of transitions)

In [ ]:
# Instantiate second lunar lander DQN agent
dqn_agent2 = DQNAgent(name, alpha, epsilon, gamma, tau, lr_period, lr_decay, batch_size, buffer_size)

# Train for specified number of episodes
dqn_agent2.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min = dqn_agent2.score_stats(verbose = True)

[Link to the video of the second DQN agent in the 512<sup>th</sup> episode](videos/dqn-agent-2-episode-512.mp4)

This seems more promising. The average is lower, but there is more exploration. We can see more fluctation in the running average, too. This exploration of choices beyond hovering in the top center has already led it toward discovering better choices. Also, the video linked above shows the agent landing instead of hovering. It still needs some refinement, but it's already much more controlled than the non-learning agents. If we run the training longer, this agent might actually find the solution, but let's try changing a few other hyperparameters before we commit the time to train longer. 

Let's try increasing the target network update rate $\tau$ next. This should speed up training, but it might also create instability.

In [ ]:
# Hyperparameters
name = "DQN Agent 3"
alpha = 0.1             # initial optimizer learning rate
epsilon = (0.99,        # epsilon-greedy max (starting value)
        0.01,           # epsilon-greedy min (ending value)
        0.001)          # exponential decay rate (smaller = slower)
gamma = 0.9             # discount rate
tau = 0.01              # target network update rate
lr_period = 100         # episodes between learning rate decreases
lr_decay = 0.1          # learning rate decay
batch_size = 128        # replay buffer batch size
buffer_size = 1000000   # replay buffer size (no. of transitions)

In [ ]:
# Instantiate lunar lander DQN agent
dqn_agent3 = DQNAgent(name, alpha, epsilon, gamma, tau, lr_period, lr_decay, batch_size, buffer_size)

# Train for specified number of episodes
dqn_agent3.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min, avg_length = dqn_agent3.score_stats(verbose = True)

[Link to the video of the third DQN agent in the 512<sup>th</sup> episode](videos/dqn-agent-3-episode-512.mp4)

The average score was lower this time. The fluctations seen in DQN Agent 2's running average have been smoothed out. This demonstrates the stabilizing effect of dampening or slowing the updates to the target network. In the video, it seems that the agent has not progressed as far in its learning compared to the previous agent. Let's go back to the previous target network update rate for now.

Let's try decreasing the greedy-epsilon decay rate next. This should encourage even more exploration.

In [ ]:
# Hyperparameters
name = "DQN Agent 4"
alpha = 0.1             # initial optimizer learning rate
epsilon = (0.99,        # epsilon-greedy max (starting value)
        0.01,           # epsilon-greedy min (ending value)
        0.0001)         # exponential decay rate (smaller = slower)
gamma = 0.9             # discount rate
tau = 0.005             # target network update rate
lr_period = 100         # episodes between learning rate decreases
lr_decay = 0.1          # learning rate decay
batch_size = 128        # replay buffer batch size
buffer_size = 1000000   # replay buffer size (no. of transitions)

In [ ]:
# Instantiate lunar lander DQN agent
dqn_agent4 = DQNAgent(name, alpha, epsilon, gamma, tau, lr_period, lr_decay, batch_size, buffer_size)

# Train for specified number of episodes
dqn_agent4.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min, avg_length = dqn_agent4.score_stats(verbose = True)

[Link to the video of the fourth DQN agent in the 512<sup>th</sup> episode](videos/dqn-agent-4-episode-512.mp4)

The average is slightly higher than the second DQN agent. There also seems to be more variation. In the video, the agent landed crashed outside the landing pad. This seems to just be prolonging the learning process.

Let's return to the second agent and see what happens when we train it longer.

In [ ]:
# Hyperparameters
name = "DQN Agent 2"
alpha = 0.1             # initial optimizer learning rate
epsilon = (0.99,        # epsilon-greedy max (starting value)
        0.01,           # epsilon-greedy min (ending value)
        0.001)          # exponential decay rate (smaller = slower)
gamma = 0.9             # discount rate
tau = 0.005             # target network update rate
lr_period = 100         # episodes between learning rate decreases
lr_decay = 0.1          # learning rate decay
batch_size = 128        # replay buffer batch size
buffer_size = 1000000   # replay buffer size (no. of transitions)

In [ ]:
# Reset the second lunar lander DQN agent
dqn_agent2 = DQNAgent(name, alpha, epsilon, gamma, tau, lr_period, lr_decay, batch_size, buffer_size)
  
# If GPU device is available
if torch.cuda.is_available():
    episodes = 1000
    print("Device: cuda")
else:
    episodes = 600
    print("Device: cpu")

# Train for specified number of episodes
dqn_agent2.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min, avg_length = dqn_agent2.score_stats(verbose = True)


[Link to the video of the second DQN agent in the 512<sup>th</sup> episode](videos/dqn-agent-2-episode-512.mp4)

Summary

## Discussion ## 

Does it include discussion (what went well or not and why), and suggestions for improvements or future work?

### Suggestions for Future Research ###

There are a number of directions for future research:

I have only explored a few optimizations in this paper. There are still many ways to improve upon the model presented here, ranging from more hyperparameter tuning to the exploration of different optimization algorithms and neural network architectures. More rigorous model testing is also needed. In addition, the Gymnasium lunar lander environment is deterministic. In such cases, Machado and his colleagues (2018) recommend introducing stochasticity with frame skipping and sticky actions that repeat with a given probability in order to explore the robustness of models in more realistic environments. Another direction for exploration is applying convolutional neural networks and comparing the results with this state feature based model.

More complex Gymnasium environments, such as the "Arcade Learning Environment" (Bellemare, 2023; Bellemare et al., 2013; Machado et al., 2018), which simulates classic Atari 2600 games (AtariAge, 2023), can also provide opportunities to explore more challenging problems. 

### Conclusion ###
This project compared four different artificial agents in a lunar lander simulation. As expected, the deep reinforcement learning agent performed better than the three non-learning agents. Of the three non-learning agents, the simple reflex agent was only slightly better than an agentless lunar lander, and acting randomly was the worst strategy. A deep Q-network with two fully-connected neural network layers was used for the learning agent. Hyperparameter tuning resulted in

Finally, it is only natural to ask: What is the value of crashing a virtual lunar lander model hundreds of times in order to eventually land successfully? There are two main reasons. First, it is better to train a far less-expensive virtual model and then transfer that learning to a real-world environment. Thus, the long-term aim is to develop the ability to transfer this virtual knowledge to a real-world environment. Second, through this project, I have gained skills and experience with deep reinforcement learning that I can build on to explore more sophisticated artificial intelligence algorithms in the future.

## References ##

_AtariAge._ (2023). [Website]. [https://atariage.com](https://atariage.com)

Bellemare, M. G. (2023). Arcade Learning Environment. [https://github.com/mgbellemare/Arcade-Learning-Environment](https://github.com/mgbellemare/Arcade-Learning-Environment)

Bellemare, M. G., Naddaf, Y., Veness, J., & Bowling, M. (2013). The Arcade Learning Environment: An evaluation platform for general agents. _Journal of Artificial Intelligence Research, 47,_ 253-279.
[https://doi.org/10.1613/jair.3912](https://doi.org/10.1613/jair.3912)

Brownlee, J. (2017, July 3). Gentle introduction to the Adam optimization algorithm for deep learning. _Machine Learning Mastery._ [https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

Goodfellow, I., Bengio, Y., & Courville, A. (2016). _Deep learning._ MIT Press. [http://www.deeplearningbook.org](http://www.deeplearningbook.org)

_Gymnasium._ (2022). Farama Foundation. [https://gymnasium.farama.org/](https://gymnasium.farama.org/)

Heintz, B. (2021, April 17). _Building models with Pytorch._ Pytorch. [https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch)

Huber, P. J. (1964). Robust estimation of a location parameter. _Annals of Statistics, 53_(1), 73–101. [https//doi.org/10.1214/aoms/1177703732](https//doi.org/10.1214/aoms/1177703732)

Karpathy, A. (2016). _Convolutional neural networks for visual recognition._ Stanford University. [https://cs231n.github.io/convolutional-networks/](https://cs231n.github.io/convolutional-networks/)

Loshchilov, I., & Hutter, F. (2019). Decoupled weight decay regularization. _International Conference on Learning Representations._[https://doi.org/10.48550/arXiv.1711.05101](https://doi.org/10.48550/arXiv.1711.05101)

Machado, M. C., Bellemare, M. G., Talvitie, E., Veness, J., Hausknecht, M., & Bowling, M. (2018). Revisiting the Arcade Learning Environment: Evaluation protocols and open problems for general agents. _Journal of Artificial Intelligence Research, 61,_ 523–562. [https://doi.org/10.1613/jair.5699](https://doi.org/10.1613/jair.5699)

Matplotlib. (2023). _API reference._ [https://matplotlib.org/stable/api/index](https://matplotlib.org/stable/api/index)

Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., Graves, A., Riedmiller, M., Fidjeland, A. K., Ostrovski, G., Petersen, S., Beattie, C., Sadik, A., Antonoglou, I., King, H., Kumaran, D., Wierstra, D., Legg, S., & Hassabis, D. (2015). Human-level control through deep reinforcement learning. _Nature, 518_, 529–533. [https://doi.org/10.1038/nature14236](https://doi.org/10.1038/nature14236)

Mnih, V., Puigdomènech Badia, A., Mirza, M., Graves, A., Harley, T., Lillicrap, T. P., Silver, D., & Kavukcuoglu, K. (2016). Asynchronous methods for deep reinforcement learning. _Proceedings of the 33rd International Conference on Machine Learning, 48,_ 1928–1937. [https://proceedings.mlr.press/v48/mniha16.html](https://proceedings.mlr.press/v48/mniha16.html)

Morales, M. (2020). _Grokking deep reinforcement learning._ Simon and Schuster.

Paszke, A., & Towers, M. (2017, April 6). _Reinforcement learning (DQN) tutorial._ Pytorch. [https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

Pytorch. (2023). _Pytorch documentation._ [https://pytorch.org/docs/stable/index.html](https://pytorch.org/docs/stable/index.html)

Reddi, S. J., Kale, S., & Kumar, S. (2019). On the convergence of adam and beyond. _International Conference on Learning Representations._[https://doi.org/10.48550/arXiv.1904.09237](https://doi.org/10.48550/arXiv.1904.09237)

Russell, S., & Norvig, R. (2022). _Artificial intelligence: A modern approach,_ (4th ed.). Pearson. 

Sutton, R. S., & Barto, A. G. (2018). _Reinforcement learning: An introduction._ MIT Press.

Tam, A. (2023, February 22). Using learning rate schedule in Pytorch training. _Machine Learning Mastery._ [https://machinelearningmastery.com/using-learning-rate-schedule-in-pytorch-training/](https://machinelearningmastery.com/using-learning-rate-schedule-in-pytorch-training/)